# Derivados Financieros 2023 - Clase 5
## Otros modelos de pricing - QuantLib

    0) PnL Explanation
    1) Montecarlo
    2) Diferencias Finitas
    3) QuantLib
    4) GUI de yapa

$$ PnL(hoy) = C(hoy)- C(ayer) = $$

$$ PnL(t=1) = C(t=1)- C(t=0) = dC(t=0) = \frac{dC}{dt}(t=0) $$

$$  \frac{dC}{dt}(t=0) =  \frac{\partial C}{\partial S}(t=0)  \frac{dS}{dt}(t=0) +  \frac{\partial C}{\partial \tau}(t=0)  \frac{d\tau}{dt}(t=0) + \frac{\partial C}{\partial \sigma}(t=0)  \frac{d\sigma}{dt}(t=0) + \frac{1}{2} \frac{\partial C}{\partial S^2}(t=0)  \left(\frac{dS}{dt}(t=0)\right)^2 \cdots $$

$$  PnL =  \frac{\partial C}{\partial S}  dS +  \frac{\partial C}{\partial \tau}  d\tau+ \frac{\partial C}{\partial \sigma}  d\sigma + \frac{1}{2} \frac{\partial C}{\partial S^2} \left(dS\right)^2 + \cdots $$


$$  PnL (hoy) =  \Delta (ayer) (S(hoy)-S(ayer)) +  \Theta(ayer)  + Vega(ayer)  (\sigma(hoy)-\sigma(ayer)) + \frac{1}{2} \Gamma  (ayer) (S(hoy)-S(ayer))^2 + \cdots $$










Este analisis nos permite por un lado verificar si nuestro modelo de precios es adecuado para el mercado y por otro, tener un control explicativo de la dinamica del precio.

Un test de PnL es un tipico ananlsis de control de riesgo. La idea es por un lado tomar 30 (u otra cantidad de dias) del pasado de datos de mercado y:

    - Por un lado calcular el PnL real, de mercado - por ejemplo call europea 


$$ C(t+1)+C(t) $$

- Por el otro con los datos de mercado de spot, volatilidad y las griegas de nuestro modelo, llegar al PnL de modelo
    - Finalemnte hacer una regrecion entre ambas
    



## Ejercicio Realizar El PnL explanation del sigueinte data historica



In [ ]:
import pandas as pd
stock_data = pd.read_pickle("./stock.pkl")


option_data = pd.read_pickle("./option.pkl")

In [ ]:
stock_data['day_num']=list(range(len(stock_data.index)))
stock_data



In [ ]:

option_data['day_num']=list(range(len(option_data.index)))
option_data['vol_imp'] = 0.0

option_data


In [ ]:
from datetime import date
business_days = [date.date() for date in option_data.index]
business_days

In [ ]:

PnL_df = pd.DataFrame(index = business_days, columns = ['PnL_Mercado', 'PnL_Modelo_Explained', 'Delta', 'Theta', 'Vega','Gamma', 'PnL_Modelo_Unexplained','day_num', 'Precio_Mercado', 'Precio_Modelo','imp_vol'])
PnL_df['day_num']=list(range(len(PnL_df.index)))

PnL_df.PnL_Mercado = 0.0
PnL_df.Precio_Modelo = 0.0

PnL_df.PnL_Modelo_Explained = 0.0
PnL_df.Delta = 0.0
PnL_df.Theta = 0.0
PnL_df.Vega = 0.0
PnL_df.Gamma = 0.0
PnL_df.PnL_Modelo_Unexplained = 0.0
PnL_df.imp_vol = 0.0



PnL_df.Precio_Mercado = option_data.CLOSE

PnL_df


In [ ]:
for i in PnL_df['day_num'].values[1:]:
    PnL_df['PnL_Mercado'].values[i] = option_data.CLOSE.values[i] - option_data.CLOSE.values[i-1]


In [ ]:
PnL_df

In [ ]:
import datetime
r = 0.025
sigma = 0.287
K = 410
div = 0.01
tipo = "C"
expiry_datetime = datetime.date(2020,10,16)

h = 0.1
h_vol = 0.01
pasos = 250


In [ ]:
import sys
sys.path.append('..')

from Codigo.opcion_europea_bs import opcion_europea_bs
from Codigo.opcion_americana_bin import opcion_americana_bin
from Codigo.vol_implicita import impvolfunc_bs
from Codigo.vol_implicita import impvolfunc_bin


In [ ]:
for i in PnL_df['day_num'].values:
    
    TTM = (expiry_datetime - PnL_df.index.values[i]).days
    T = TTM/365.0
    S = stock_data.CLOSE.values[i]
    
    C = opcion_europea_bs(tipo, S, K, T, r, sigma, div)
    
    
    PnL_df['Precio_Modelo'].values[i] = C
    
    
    PnL_df['imp_vol'].values[i] = impvolfunc_bs(tipo, stock_data.CLOSE.values[i], K, T, r, option_data.CLOSE.values[i] , div)
    
    sigma = PnL_df['imp_vol'].values[i]
    
    
    C_Sup = opcion_europea_bs(tipo, S + h, K, T, r, sigma, div)
    C_Sdown = opcion_europea_bs(tipo, S - h, K, T, r, sigma, div)
    
    PnL_df['Delta'].values[i] = (C_Sup - C_Sdown) / (2*h) 
    
    PnL_df['Gamma'].values[i] = (C_Sup - 2 * C + C_Sdown) / (h**2) 
    
    C_prox = opcion_europea_bs(tipo, S, K, T-1/252.0, r, sigma, div)
    
    PnL_df['Theta'].values[i] = (C_prox - C)
    

    
    C_volup = opcion_europea_bs(tipo, S, K, T, r, sigma + h_vol, div)
    C_voldown = opcion_europea_bs(tipo, S , K, T, r, sigma - h_vol, div)
    
    PnL_df['Vega'].values[i] = (C_volup - C_voldown) / (2*h_vol) 
    
    


In [ ]:
PnL_df

In [ ]:
for i in PnL_df['day_num'].values[1:]:
    #delta
    PnL_df.PnL_Modelo_Explained.values[i] = PnL_df.Delta.values[i-1] * (stock_data.CLOSE.values[i] -stock_data.CLOSE.values[i-1])#theta
    #gamma
    #PnL_df.PnL_Modelo_Explained.values[i] += 1/2* PnL_df.Gamma.values[i-1] * (stock_data.CLOSE.values[i] -stock_data.CLOSE.values[i-1])**2 #gamma
    #theta
    PnL_df.PnL_Modelo_Explained.values[i] += PnL_df.Theta.values[i-1]  
    #vega
    #PnL_df.PnL_Modelo_Explained.values[i] += PnL_df.Vega.values[i-1] * (PnL_df.imp_vol.values[i] - PnL_df.imp_vol.values[i-1])
  
    PnL_df.PnL_Modelo_Unexplained.values[i] = PnL_df.PnL_Mercado.values[i] - PnL_df.PnL_Modelo_Explained.values[i]
    
    

In [ ]:
PnL_df['PnL_Modelo'] = 0.0
for i in PnL_df['day_num'].values[1:]:
    PnL_df['PnL_Modelo'].values[i] = PnL_df.Precio_Modelo.values[i]-PnL_df.Precio_Modelo.values[i-1]
PnL_df

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
x = np.linspace(-20, 20, 1000)

plt.plot(x, x, 'r-')
plt.plot(PnL_df['PnL_Modelo'].values, PnL_df['PnL_Modelo_Explained'].values, 'o')
plt.title('PnL Attribute')
plt.xlabel('PnL Modelo')
plt.ylabel('PnL Explained Modelo')

plt.show()

In [ ]:
x = np.linspace(-20, 20, 1000)

plt.plot(x, x, 'r-')
plt.plot(PnL_df['PnL_Modelo'].values, PnL_df['PnL_Modelo_Explained'].values, 'o')
plt.title('PnL Attribute')
plt.xlabel('PnL Modelo')
plt.ylabel('PnL Explained Modelo')

plt.xlim(-5,5)
plt.ylim(-5,5)


plt.show()

## 1) Metodo de Montecarlo

In [ ]:
from Codigo.opcion_europea_bs import opcion_europea_bs
from Codigo.opcion_europea_mc import opcion_europea_mc

import math

opcion_europea_mc
Def
    Calculador del precio de una opcion Europea con el modelo de MonteCarlo
Inputs
    - tipo : string - Tipo de contrato entre ["CALL","PUT"]
    - S : float - Spot price del activo
    - K : float - Strike price del contrato
    - T : float - Tiempo hasta la expiracion (en años)
    - r : float - Tasa 'libre de riesgo' (anualizada)
    - sigma : float - Volatilidad implicita (anualizada)
    - div : float - Tasa de dividendos continuos (anualizada)
    - pasos : int - Cantidad de caminos de montecarlo
Outputs
    - precio_MC: float - Precio del contrato


In [ ]:
#Inicializo los parametros


tipo = "C"
S = 100
K = 100
T = 1
r = 0.05
sigma = 0.25
div = 0

In [ ]:
import numpy as np
#Tomo un vector de normales aleatorias
pasos = 50
z = np.random.normal(0,1,pasos)





In [ ]:
import matplotlib.pyplot as plt 
import math
pasos = 500
z = np.random.normal(0,1,pasos)
W=np.zeros(pasos)
W[0] = 0
for pasos in range(1,len(z)):
    W[pasos] =W[pasos-1]+z[pasos]

plt.plot(W)


In [ ]:
pasos = 50
z = np.random.normal(0,1,pasos)
S = np.zeros(pasos)
S[0] = 100
mu = .5
sigma = .25

for pasos in range(1,len(z)-1):
    S[pasos] =S[0]+ math.exp((mu-1/2*sigma*sigma)+sigma*z[pasos])

plt.plot(S)



In [ ]:
T = 1        # Tiempo hasta madurez de la opción (1 año)
mu = 0.2    # Tasa de crecimiento esperada del activo
sigma = 0.2 # Volatilidad del proceso
S_0 = 100      # Condición inicial (precio inicial)
dt = 0.0001  # Paso de tiempo (en la misma unidad que T).
M = 50      # Cantidad de trayectorias a simular en 1 pasada.
N = round(T/dt)
t = np.linspace(0, T, N)

# Configuración del gráfico
fig = plt.figure(figsize=(12,7), dpi = 300)
ax = fig.add_subplot()
ax.set_title(f'$ S_0$ = {S_0}, \t $\mu$ = {100*mu}%, \t $\sigma$ = {sigma}, \t $T$ = {T} \n {M} trayectorias')

# Simulación y gráfico
for _ in range(M):
    W = np.random.standard_normal(size = N) 
    W = np.cumsum(W)*np.sqrt(dt)   ### proceso de Wiener ###
    X = (mu-0.5*sigma**2)*t + sigma*W 
    S = S_0*np.exp(X)   ### movimiento Browniano geométrico ###
    ax.plot(t, S, lw = 0.4)
ax.grid(True, linestyle='-')
ax.set_xlabel('Tiempo [años]')
ax.set_ylabel('Precio [USD]')
#plt.show()


In [ ]:
opcion = np.zeros(pasos) #En el ppt. C = C_i

In [ ]:
for i in range(0,pasos):
    if tipo == "C":
        S_i = S_0 * math.exp((r-div - 0.5 * math.pow(sigma,2)) * T + sigma * math.sqrt(T)  * z[i])
        payoff = max( 0 , S_i - K) # max(0,S_i-K)
    elif tipo == "P":
        payoff = max(0, K - S_0 * math.exp((r-div - 0.5 * math.pow(sigma, 2)) * T + sigma * math.sqrt(T) * z[i]) )
    opcion[i] = math.exp(-r * T) * payoff #C_i


In [ ]:
precio_MC = np.mean(opcion)

precio_MC

desvio = math.sqrt(np.var(opcion)/pasos)

print("precio",precio_MC)
print("desvio", desvio)

### Convergencia del metodo

In [ ]:
tipo = "C"
S = 100
K = 100
T = 1
r = 0.05
sigma= 0.25
div =0.0

pasos_vec = [1,2,3,4,5,6,7,8,9,10,12,14,16,18,20,25,30,40,50,75,100,125,150,200,250,300, 500, 5000, 20000]      
precios = np.zeros(len(pasos_vec))

for index in range(len(pasos_vec)):
    precios[index] = opcion_europea_mc(tipo, S, K, T, r, sigma, div, pasos_vec[index])
 
precio_BS = opcion_europea_bs(tipo, S, K, T, r, sigma, div)

In [ ]:
from matplotlib import pyplot as plt
plt.axhline(y=precio_BS, color = 'black', linestyle='--', label='Precio BlackSholes')
plt.plot(pasos_vec,precios,'*', label='Precio Montecarlo')

plt.legend()

plt.xlabel('Caminos de Montecarlo')
plt.ylabel('Precio de la opcion')
plt.title('Precio de una opcion a tiempo inicial con respecto al numero caminos de montecarlo')

plt.show()

In [ ]:
print(precio_BS, precios)



# 2) Metodo de diferencias finitas

In [ ]:
from Codigo.opcion_americana_fd import opcion_americana_fd
from Codigo.opcion_europea_fd import opcion_europea_fd

from scipy.interpolate import interp1d

opcion_europea_fd
Def
    Calculador del precio de una opcion Europea con el modelo de Diferencias Finitas (metodo explicito)
Inputs
    - tipo : string - Tipo de contrato entre ["CALL","PUT"]
    - S : float - Spot price del activo
    - K : float - Strike price del contrato
    - T : float - Tiempo hasta la expiracion (en años)
    - r : float - Tasa 'libre de riesgo' (anualizada)
    - sigma : float - Volatilidad implicita (anualizada)
    - div : float - Tasa de dividendos continuos (anualizada)
Outputs
    - precio_FD: float - Precio del contrato

In [ ]:
tipo = "C"
S = 100
K = 100
T = 1
r = 0.05
sigma= 0.25
div =0.0

In [ ]:
#Hadrcode de la grilla de diferencias finitas
M = 160
N = 1600

#M = 6
#N = 4


dS = 2 * S / M
dt = T / N

In [ ]:
# Grilla de spots y tiempos
S_vec = np.linspace(0, 2*S, M+1)
t_vec = np.linspace(0, T, N+1)



In [ ]:
# Armado de la matriz tridiagonal
j = np.arange(1,M)
j2 = np.zeros(M-1)
aj = np.zeros(M-1)
bj = np.zeros(M-1)
cj = np.zeros(M-1)

for index in range(0,M-1):
    sigma2 = sigma*sigma
    j2[index] = j[index] * j[index]
    aj[index] = 0.5 * dt * (sigma2 * j2[index]- (r-div) * j[index])
    bj[index] = 1-dt * (sigma2 * j2[index] + r)
    cj[index] = 0.5 * dt * (sigma2 * j2[index] + (r-div) * j[index])

In [ ]:
# Matriz tridiagonal

A = np.diag(bj)
for index in range(0, M - 2):
    A[index + 1, index] = aj[index + 1]  # terms below the diagonal
    A[index, index + 1] = cj[index]  # terms above the diagonal
    


In [ ]:
# Matriz de precios de la opcion
opcion_precios = np.zeros((M+1,N+1)) #V

In [ ]:
#Condiciones de contorno

# Condicion final - Payoff

if tipo == "C":
    opcion_precios[:,-1] = np.maximum(S_vec - K, 0)
elif tipo == "P":
    opcion_precios[:,-1] = np.maximum(K - S_vec, 0)

In [ ]:
# Casos limite en S=0 y S~inf

if tipo == "C":
    opcion_precios[0, :] = 0
    opcion_precios[-1, :] = S_vec[-1]*np.exp(-div*np.flip(t_vec)) - K * np.exp(-r*np.flip(t_vec))
elif tipo == "P":
    opcion_precios[0, :] = K * np.exp(-r * np.flip(t_vec))
    opcion_precios[-1, :] = 0 #K * np.exp(-r * np.flip(t_vec))

In [ ]:
    
# Calculo en el interior
# variable auxiliar para sumar en la primer y ultimo fila
constantes_bordes = np.array((aj[0], cj[-1]))
for i in list(reversed(range(0,N))):

    opcion_precios[1:M,i] = A @ opcion_precios[1:M,i+1]
    #Offset the first and last terms
    opcion_precios[[1,M-1],i] = opcion_precios[[1,M-1],i] + constantes_bordes * opcion_precios[[0, M],i+1];

In [ ]:
from matplotlib import pyplot as plt

plt.plot(S_vec,opcion_precios[:,0])

plt.show()

In [ ]:
#En este punto ya esta TODA la grilla, ahora calculo lo requerido

f = interp1d(S_vec,opcion_precios[:,0])

S=100
precio_FD = float(f(S))




precio_FD


In [ ]:
from mpl_toolkits import mplot3d

X, Y = np.meshgrid(t_vec, S_vec)
Z = opcion_precios

In [ ]:
print('X', np.shape(X))
print('Y', np.shape(Y))
print('Z', np.shape(Z))

In [ ]:
fig = plt.figure()

ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z, cmap='coolwarm')#, edgecolor='none')#rstride=1, cstride=1,
                

ax.set_title('Precio de la opcion');
ax.set_xlabel('Tiempo - t')
ax.set_ylabel('Precio del subyacente - S')
ax.set_zlabel('Precio de la opcion')

plt.show()

# 3) QuantLib o Introduccion a una Verdadera plataforma de pricing/riesgo

In [ ]:
pip install QuantLib

In [ ]:
from Codigo.utils_plots import *
import QuantLib as ql

### Opciones

### Supongamos una opcion de AAPL con precio strike de 450 expirando el 16 de octubre de 2020. Supongamos que el spot es 440. La vol implicita es sabida que es 31%, y tiene un dividendo de 0.75%. Valuemos la opcion al dia de hoy (10 de Agosto 2020)

#### Definimos los inputs

In [ ]:
S = 456.30
K = 460
sigma = 0.3394 # the historical vols for a year
div =  0.075
r = 0.0013

option_type = ql.Option.Call

option_type

#### Para las fechas QuantLib es mas preciso. Pide convenciones y calendarios

In [ ]:
maturity_date = ql.Date(16, 10, 2020)
calculation_date = ql.Date(6, 8, 2020)

day_count = ql.Actual365Fixed()


ql.Argentina()
calendar = ql.UnitedStates(ql.UnitedStates.NYSE)

ql.UnitedStates
ql.Settings.instance().evaluationDate = calculation_date


maturity_date

#### Aqui construimos a la opcion europea

In [ ]:
payoff = ql.PlainVanillaPayoff(option_type, K)

exercise = ql.EuropeanExercise(maturity_date)
european_option = ql.VanillaOption(payoff, exercise)



#### Aca constuimos el proceso de Black Sholes Merton

##### El objeto spot

In [ ]:
spot_obj = ql.QuoteHandle(ql.SimpleQuote(S))

In [ ]:
spot_obj.value()

##### El objeto curva de descuento (flat en este caso, constante)

In [ ]:
rate_obj = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, 
                                                            r, 
                                                    day_count))

##### El objeto curva de dividendos (flat en este caso, constante)

In [ ]:
dividend_obj = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, 
                                                      div, 
                                                      day_count))

##### El objeto volatilidad (flat en este caso, constante)

In [ ]:
vol_obj = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, 
                                                                 calendar, 
                                                                 sigma, 
                                                             day_count))

In [ ]:
##### El proceso propiamente dicho

In [ ]:
bsm_process = ql.BlackScholesMertonProcess(spot_obj, 
                                        dividend_obj, 
                                        rate_obj, 
                                        vol_obj)

## Modelos de precio

###  `AnalyticEuropeanEngine` (Black Scholes)

In [ ]:
european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))

bs_price = european_option.NPV()
print("El precio teorico usando el modelo de BS es: ", bs_price)

###  `FdBlackScholesVanillaEngine` (Diferencias Finitas)

In [ ]:
european_option.setPricingEngine(ql.FdBlackScholesVanillaEngine(bsm_process))

fd_price = european_option.NPV()
print("El precio teorico usando el modelo de Dif. Finitas es: ", fd_price)

###  `MCEuropeanEngine` (Montecarlo)

In [ ]:
european_option.setPricingEngine(ql.MCEuropeanEngine(bsm_process, "PseudoRandom", timeSteps=20, requiredSamples=500000))

mc_price = european_option.NPV()
print("El precio teorico usando el modelo de MonteCarlo es: ",  mc_price)

In [ ]:
european_option.errorEstimate()

###  `BinomialVanillaEngine` (Binomial)

In [ ]:
european_option.setPricingEngine(ql.BinomialVanillaEngine(bsm_process, "crr", 1000))

bin_price = european_option.NPV()
print("El precio teorico usando el modelo Binomial es: ",  bin_price)

# Comparacion de TODOS* los modelos vistos (caso de juguete)

### Fijo los parametros

In [ ]:
S = 100
K = 100
sigma = 0.25 # the historical vols for a year
div =  0.0
r = 0.05

tipo = 'C'

option_type = ql.Option.Call

maturity_date = ql.Date(10, 8, 2021)
calculation_date = ql.Date(10, 8, 2020)
day_count = ql.Actual365Fixed()
nyse = ql.UnitedStates.NYSE

calendar = ql.UnitedStates(nyse)
ql.Settings.instance().evaluationDate = calculation_date

T = 1



## Pricers vistos en clase

### Importo los pricers vistos en clase

In [ ]:
from Codigo.opcion_europea_bs import opcion_europea_bs
from Codigo.opcion_europea_mc import opcion_europea_mc
from Codigo.opcion_europea_fd import opcion_europea_fd
from Codigo.opcion_europea_bin import opcion_europea_bin

import timeit

### Corro cada uno de los pricers (les tomo el tiempo de ejecucion tambien)

In [ ]:
start = timeit.default_timer()
precio_bs = opcion_europea_bs(tipo, S, K, T, r, sigma, div)
end = timeit.default_timer()
tiempo_bs = end - start


start = timeit.default_timer()
precio_bin = opcion_europea_bin(tipo, S, K, T, r, sigma, div, 5000)
end = timeit.default_timer()
tiempo_bin = end - start

start = timeit.default_timer()
precio_mc = opcion_europea_mc(tipo, S, K, T, r, sigma, div, 5000)
end = timeit.default_timer()
tiempo_mc = end - start

start = timeit.default_timer()
precio_fd = opcion_europea_fd(tipo, S, K, T, r, sigma, div)
end = timeit.default_timer()
tiempo_fd = end - start

## Precios QuantLib

### Seteo lo necesario en QuantLib

In [ ]:
payoff = ql.PlainVanillaPayoff(option_type, K)
exercise = ql.EuropeanExercise(maturity_date)
european_option = ql.VanillaOption(payoff, exercise)

spot_obj = ql.QuoteHandle(ql.SimpleQuote(S))

rate_obj = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, 
                                                            r, 
                                                    day_count))

dividend_obj = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, 
                                                      div, 
                                                      day_count))

vol_obj = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, 
                                                                 calendar, 
                                                                 sigma, 
                                                             day_count))

bsm_process = ql.BlackScholesMertonProcess(spot_obj, 
                                        dividend_obj, 
                                        rate_obj, 
                                        vol_obj)



### Corro cada uno de los pricers (les tomo el tiempo de ejecucion tambien)

In [ ]:

start = timeit.default_timer()
european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
bs_price = european_option.NPV()
end = timeit.default_timer()
time_bs = end - start

start = timeit.default_timer()
european_option.setPricingEngine(ql.FdBlackScholesVanillaEngine(bsm_process))
fd_price = european_option.NPV()
end = timeit.default_timer()
time_fd = end - start

start = timeit.default_timer()
european_option.setPricingEngine(ql.MCEuropeanEngine(bsm_process, "PseudoRandom", timeSteps=20, requiredSamples=5000))
mc_price = european_option.NPV()
end = timeit.default_timer()
time_mc = end - start

start = timeit.default_timer()
european_option.setPricingEngine(ql.BinomialVanillaEngine(bsm_process, "crr", 5000))
bin_price = european_option.NPV()
end = timeit.default_timer()
time_bin = end - start



### Imprimo TODO

In [ ]:
print("Precio modelo Europea BS visto en clase:", precio_bs)
print("Tiempo modelo Europea BS visto en clase:", tiempo_bs)

print("Precio modelo Europea Binomial visto en clase:", precio_bin)
print("Tiempo modelo Europea Binomial visto en clase:", tiempo_bin)

print("Precio modelo Europea Montecarlo visto en clase:", precio_mc)
print("Tiempo modelo Europea Montecarlo visto en clase:", tiempo_mc)

print("Precio modelo Europea Dif. Finitas visto en clase:", precio_fd)
print("Tiempo modelo Europea Dif. Finitas visto en clase:", tiempo_fd)


print("Precio modelo Europea BS QuantLib", bs_price)
print("Tiempo modelo Europea BS QuantLib:", time_bs)

print("Precio modelo Europea Binomial QuantLib:", bin_price)
print("Tiempo modelo Europea Binomial QuantLib:", time_bin)

print("Precio modelo Europea Montecarlo QuantLib:", mc_price)
print("Tiempo modelo Europea Montecarlo QuantLib:", time_mc)

print("Precio modelo Europea Dif. Finitas QuantLib:", fd_price)
print("Tiempo modelo Europea Dif. Finitas QuantLib:", time_fd)

### Tabla de precios por modelo [NO ACTUALIZADA]

| Precio | Clase | QuantLib |
| --- | --- | --- |
| BS | 12.336 | 12.336 |
| Binomial | 12.33355 | 12.33347 |
| Montecarlo | _12.33747_ | _12.38694_ |
| Dif.Finitas | 12.33380 | 12.34399 |

### Tabla de tiempos de ejecucion por modelo**

| Tiempo | Clase | QuantLib |
| --- | --- | --- |
| BS | **0.00033** | **0.00023** |
| Binomial | 17.38176 | **0.06012** |
| Montecarlo | **0.72694** | 3.03739 |
| Dif.Finitas | 0.046359 | **0.001721** |

### Opciones Americanas


In [ ]:
S = 100
K = 100
sigma = 0.25 # the historical vols for a year
div =  0.0
r = 0.05
tipo = 'C'

option_type = ql.Option.Put

maturity_date = ql.Date(10, 8, 2021)
calculation_date = ql.Date(10, 8, 2020)
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates(nyse)
ql.Settings.instance().evaluationDate = calculation_date

T = 1


In [ ]:
payoff = ql.PlainVanillaPayoff(option_type, K)

am_exercise = ql.AmericanExercise(calculation_date, maturity_date)
american_option = ql.VanillaOption(payoff, am_exercise)

In [ ]:
spot_obj = ql.QuoteHandle(ql.SimpleQuote(S))

rate_obj = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, 
                                                            r, 
                                                    day_count))

dividend_obj = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, 
                                                      div, 
                                                      day_count))

vol_obj = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, 
                                                                 calendar, 
                                                                 sigma, 
                                                             day_count))

bsm_process = ql.BlackScholesMertonProcess(spot_obj, 
                                        dividend_obj, 
                                        rate_obj, 
                                        vol_obj)

In [ ]:
# No hay Black Scholes
american_option.setPricingEngine(ql.BinomialVanillaEngine(bsm_process, "crr", 1000))
bin_price_ame = american_option.NPV()

american_option.setPricingEngine(ql.FdBlackScholesVanillaEngine(bsm_process))
fd_price_ame = american_option.NPV()

american_option.setPricingEngine(ql.MCAmericanEngine(bsm_process, "PseudoRandom", timeSteps=20, requiredSamples=500000))
mc_price_ame = american_option.NPV()



In [ ]:
american_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
bin_price_ame = american_option.NPV()

In [ ]:
from Codigo.opcion_americana_bin import opcion_americana_bin
from Codigo.opcion_americana_fd import opcion_americana_fd

In [ ]:
tipo = "P"
precio_bin_ame = opcion_americana_bin(tipo, S, K, T, r, sigma, div, 1000)
precio_fd_ame = opcion_americana_fd(tipo, S, K, T, r, sigma, div)


In [ ]:
print("Precio modelo Amercana Binomial visto en clase:", precio_bin_ame)
print("Precio modelo Amercana Dif. Finitas visto en clase:", precio_fd_ame)

print("Precio modelo Amercana Binomial QuantLib:", bin_price_ame)
print("Precio modelo Amercana Montecarlo QuantLib:", mc_price_ame)
print("Precio modelo Amercana Dif. FinitasQuantLib:", fd_price_ame)